## Metasploit Payload Size
#### Or: "How big can my stagers be, really?"

The idea here is to parse through the Metasploit Project's available exploits to determine what the distribution of payload sizes is.

This can help make decisions for stager size optimization - if I have a great idea for a stager (or other exploit payload), but can't make it any smaller than 1k, is it worth it? What if it's 2k? And so on.

As it turns out, payloads over 2kb work with less than 20% of available exploits, and payloads over 1kb only work with about 60% - if you can't make your stager under 2k, you shouldn't expect to be able to use it very often at all.

In [163]:
%matplotlib inline

import os
import re
import sys
import numpy as np
import matplotlib

matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

In [164]:
# Set up a path to the Metasploit project's code.
basepath = os.path.join('/', 'home', 'dnelson', 'projects', 'msf-stats')
rootdir = os.path.join(basepath, 'metasploit-framework', 'modules', 'exploits')
rootdir

'/home/dnelson/projects/msf-stats/metasploit-framework/modules/exploits'

In [165]:
# Iterate through every exploit module, searching for the amount of space that exploit provides to fit a payload in.
all_sizes = []
for folder, subs, files in os.walk(rootdir):
    for filename in files:
        with open(os.path.join(folder, filename), 'r') as sploit:
            #print("parsing " + filename + "...")
            text = sploit.read()
            # remove all whitespace
            text = ''.join(text.split())
            space = re.search("\'Space\'=>(\d+)\,", text)
            # Note that if no payload size limit is specified, we simply ignore that module
            if space:
                all_sizes.append(int(space.group(1)))

print("Modules processed: " + str(len(all_sizes)))

Modules processed: 950


In [166]:
sorted_sizes = np.sort(all_sizes)
cumulative = np.cumsum(sorted_sizes)

print(sorted_sizes)

# looks to me like we should exclude that last one, since it's waaaaaaay larger than the rest
sorted_sizes = sorted_sizes[:-1]

[     100      127      128      128      130      153      160      164
      175      200      200      200      200      200      210      212
      213      220      227      228      232      232      236      250
      250      250      250      250      250      253      255      255
      256      256      256      256      256      260      284      296
      300      300      300      300      300      300      310      336
      336      340      344      344      344      350      370      370
      373      375      380      384      390      392      400      400
      400      400      400      400      400      400      400      400
      400      400      400      400      400      400      400      400
      400      400      400      400      400      400      400      400
      400      400      400      410      417      424      424      434
      440      444      450      450      450      450      450      450
      460      466      472      475      476      

(0, 949)

In [178]:
# Simply plot a cumulative sum of payload sizes
plt.plot(cumulative, linewidth=3, color="black")

plt.ylabel("Payload space (bytes, to 1 MB)", size=16)
plt.ylim((0,1000000)) # only go out to 1 MB

plt.xlabel("Number of exploit modules which will NOT work", size=16)
plt.xlim(0,len(sorted_sizes))

This is a fairly simple view on the data: plot each exploit against its payload space. To interpret this, imagine that at any point on the x axis, that module, and every module to the left of it, will work for the given y value (payload size) or less.

In [179]:
# Plot our sorted sizes against a fraction, 0 to 1, of all exploits.
plt.plot(sorted_sizes, np.linspace(1,0,len(sorted_sizes)), linewidth=3, color='black')
plt.xlim((0,10000))

plt.xlabel("Payload size (bytes)", size=16)
plt.ylabel("Fraction of exploits which\n accept that payload size", size=16)

# Plot some vertical lines for emphasis; a red line at 512 bytes, green at 1024, and blue at 2048.
plt.axvline(512, color='red', linewidth=2)
plt.axvline(1024, color='green', linewidth=2)
plt.axvline(2048, color='blue', linewidth=2)

This second view is a lot more useful, as it displays the payload size against the fraction of exploits which will work (or not work) for that size. It looks like any payload over 2048 bytes will only work with about 20% of exploits - a little less, in fact! If any of your stagers are just barely above 1024 bytes, it's well worth the effort to trim those last few bytes. Almost a quarter of exploits available in Metasploit have a payload size cutoff at 1024 bytes.

This chart can also be read as a probability: if I want to send a 1000 byte payload, and I pick an exploit at random (or, I find an vulnerable host at random), I have about a 60% chance that the exploit I end up with will be able to accomodate that payload. If my payload is 500 bytes, that probability becomes more than 90%.

In [180]:
plt.hist(sorted_sizes, 400)

plt.xlim((0,35000))
plt.yscale('log')

plt.xlabel("Payload Size (bytes)", size=16)
plt.ylabel("Number of Exploits, Log Scale", size=16)

The humble histogram finishes out our exploration today - note that it's on a log scale. This is significantly less useful than the above charts, since payload size is a cumulative number (i.e. smaller payloads still work in exploits with more than enough space for them), but this view is interesting in that it shows us where there are large clusters of exploits accepting a certain payload size.